In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
file_path = "/content/Load_Data_in_kW[1].xlsx"
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="Sheet1")

df.columns = ["Load_kW"]  # Rename column

df["Timestamp"] = pd.date_range(start="2025-01-01", periods=len(df), freq="H")
df.set_index("Timestamp", inplace=True)

df["Load_kW_lagged"] = df["Load_kW"].shift(168)  # Create lag feature
df.dropna(inplace=True)

# Define feature and target
X = df["Load_kW_lagged"].values.reshape(-1, 1)
y = df["Load_kW"].values.reshape(-1, 1)

# Normalize data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=42, shuffle=False)

# Hyperparameters
eta = 0.1  # Learning rate
epochs = 100
epsilon = 1e-8  # Prevent division by zero

# Initialize parameters
m, c = 1.0, -1.0
G_m, G_c = 0, 0  # Initialize accumulators

# Training loop
for epoch in range(epochs):
    y_pred = m * X_train + c
    error = y_pred - y_train

    grad_m = np.mean(2 * error * X_train)
    grad_c = np.mean(2 * error)

    G_m += grad_m ** 2
    G_c += grad_c ** 2

    m -= (eta / (np.sqrt(G_m) + epsilon)) * grad_m
    c -= (eta / (np.sqrt(G_c) + epsilon)) * grad_c

# Predictions
y_train_pred = m * X_train + c
y_test_pred = m * X_test + c

# Compute errors
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Display results
print("Training Errors:")
print(f"MSE: {mse_train:.6f}, RMSE: {rmse_train:.6f}, MAE: {mae_train:.6f}")
print("Test Errors:")
print(f"MSE: {mse_test:.6f}, RMSE: {rmse_test:.6f}, MAE: {mae_test:.6f}")

Training Errors:
MSE: 0.002213, RMSE: 0.047039, MAE: 0.029100
Test Errors:
MSE: 0.000738, RMSE: 0.027158, MAE: 0.027133


<ipython-input-4-4b82ce309876>:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["Timestamp"] = pd.date_range(start="2025-01-01", periods=len(df), freq="H")
